# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

**Шаги для выбора локации:**

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Описание данных.**
**Признаки:**

`id` - уникальный идентификатор скважины

`f0, f1, f2` - три признака точек (неважно, что они означают, но сами признаки значимы)

**Целевой признак:**

`product` - объём запасов в скважине (тыс. баррелей)

**Условия задачи:**

* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).

* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

* Бюджет на разработку скважин в регионе — 10 млрд рублей.

* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

* Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

**Используемые инструменты:**

pandas, numpy, warnings, sklearn.

## Загрузка и подготовка данных

In [1]:
# импортируем все библиотеки и задаём псевдослучайность в начале ноутбука
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

STATE = np.random.RandomState(12345)

In [2]:
# прочитаем все файлы и сохраним в переменных
df0 = pd.read_csv('/datasets/geo_data_0.csv')
df1 = pd.read_csv('/datasets/geo_data_1.csv')
df2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
# для удобства сохраним в переменной список всех фреймов
data = [df0, df1, df2]

In [4]:
# изучим данные
for i in range(3):
    print('Общая информация о фрейме:')
    display(data[i].info())
    display(data[i].head())
    display(data[i].describe().T)
    print('Проверка на пропуски:')
    display(data[i].isna().sum())
    print('Проверка на дубликаты:')
    display(data[i].duplicated().sum())
    print('Корреляция:')
    display(data[i].corr())
    print('-' * 100)

Общая информация о фрейме:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,count,mean,std,min,25%,50%,75%,max
f0,100000.0,0.500419,0.871832,-1.408605,-0.072580,0.502360,1.073581,2.362331
f1,100000.0,0.250143,0.504433,-0.848218,-0.200881,0.250252,0.700646,1.343769
f2,100000.0,2.502647,3.248248,-12.088328,0.287748,2.515969,4.715088,16.003790
product,100000.0,92.500000,44.288691,0.000000,56.497507,91.849972,128.564089,185.364347


Проверка на пропуски:


...

Проверка на дубликаты:


0

Корреляция:


,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


----------------------------------------------------------------------------------------------------


In [5]:
# дропнем ненужный в исследовании признак
for i in range(3):
    data[i] = data[i].drop(columns=['id'])
    print(data[i].columns.unique())

Index(['f0', 'f1', 'f2', 'product'], dtype='object')
Index(['f0', 'f1', 'f2', 'product'], dtype='object')
Index(['f0', 'f1', 'f2', 'product'], dtype='object')


Все три фрейма одинаковы и состоят из 10е4 строк и 5 стобцов. Все стобцы в нужном виде, выбросов, дубликатов и пропусков нет. Фреймы отлично подготовлены для исследования. Заметна сильная положительная корреляция признака f2 и целевого признака во втором фрейме. Удалили признак id из всех фреймов, т.к. эти данные не несут никакой информативности для нашего исследования.

## Обучение и проверка модели

In [6]:
# создадим пустые списки для целевого признака и предсказаний
targets_valid = []
predicts = []
# напишем функцию для разбития на выборки и обучения моделей для всех фреймов
def split(df):
# разделим на признаки
    features = df.drop(["product"], axis=1)
    target = df["product"]
# разобьём на выборки в соотношении 75/25 
    features_train, features_valid, target_train, target_valid = \
        train_test_split(features, target, test_size=0.25, random_state=STATE)
# нормализуем признаки
    scaler = StandardScaler()
    scaler.fit(features_train)
    features_train_0 = scaler.transform(features_train)
# обучим модель и выведем rmse и точность модели
    model = LinearRegression()
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    rmse = mean_squared_error(target_valid, predict)**0.5
    print('Среднеквадратичное отклонение равно: %.2f' % rmse)
    result = model.score(features_valid, target_valid)
    print('Точность модели равна: {:.2%}'.format(result))
# добавим результаты в отдельный список  
    targets_valid.append(target_valid.reset_index(drop=True))
    predicts.append(predict)
# дополнительно выведем размеры наших выборок    
    print('Размер обущающей выборки:')
    print(features_train.shape)
    print(target_train.shape)
    print('Размер валидационной выборки:')
    print(features_valid.shape)
    print(target_valid.shape)

In [7]:
for i in range(3):
    print('Данные по df', i)
    split(data[i])
    print('-' * 100)

Данные по df 0
Среднеквадратичное отклонение равно: 37.58
Точность модели равна: 27.99%
Размер обущающей выборки:
(75000, 3)
(75000,)
Размер валидационной выборки:
(25000, 3)
(25000,)
----------------------------------------------------------------------------------------------------
Данные по df 1
Среднеквадратичное отклонение равно: 0.89
Точность модели равна: 99.96%
Размер обущающей выборки:
(75000, 3)
(75000,)
Размер валидационной выборки:
(25000, 3)
(25000,)
----------------------------------------------------------------------------------------------------
Данные по df 2
Среднеквадратичное отклонение равно: 39.96
Точность модели равна: 20.26%
Размер обущающей выборки:
(75000, 3)
(75000,)
Размер валидационной выборки:
(25000, 3)
(25000,)
----------------------------------------------------------------------------------------------------


Как мы видим, модель, обученная на данных из второго фрейма, имеет очень высокую точность, но низкое rmse, что может быть связано с тем, что у данных в этом фрейме очень малый разброс. Также, мы сохранили полученные по валидационным выборкам предсказания в отдельную переменную.

## Подготовка к расчёту прибыли

In [8]:
# сохраним ключевые значения в переменных
POINTS = 500  
BEST_POINTS = 200    
BUDGET = 10e9
COST_1_BARREL = 45e4 

In [9]:
# рассчитаем объём нефти для безубыточной работы
print("Объём нефти в скважине, требуемый для безубыточной работы: {0:.0f} тыс. баррелей.". \
      format(BUDGET / COST_1_BARREL /BEST_POINTS))

Объём нефти в скважине, требуемый для безубыточной работы: 111 тыс. баррелей.


In [10]:
# узнаем средний объём нефти в каждом регионе
for i in range(3):
    print('Объём нефти в регионе', i,': {0:.1f}'.format(data[i]['product'].mean()), 'тыс. баррелей')

Объём нефти в регионе 0 : 92.5 тыс. баррелей
Объём нефти в регионе 1 : 68.8 тыс. баррелей
Объём нефти в регионе 2 : 95.0 тыс. баррелей


In [11]:
# напишем функцию для подсчёта выручки
def profit(target_valid, predictions_valid, count):
    
    target = target_valid.reset_index(drop = True)
    predictions = pd.Series(predictions_valid, index = target.index)
    best_points = target[predictions.sort_values(ascending = False).index][:count]
    
    return (best_points.sum() * COST_1_BARREL - BUDGET) / 1000000

Объём нефти в каждом регионе заметно ниже, требуемого для безубыточной работы. Следовательно, вести разработку всех скважин в регионах, нерентабельно. Далее в расчётах будем использовать лишь двести скважин.

## Расчёт прибыли и рисков 

In [12]:
# рассчитаем прибыль с каждого региона
for i, x in zip(targets_valid, predicts):
    print('Прибыль с региона: {0:.1f}'.format(profit(i, x, BEST_POINTS)), 'млн. руб.')

Прибыль с региона: 3320.8 млн. руб.
Прибыль с региона: 2415.1 млн. руб.
Прибыль с региона: 2539.9 млн. руб.


In [13]:
# напишем функцию для bootstrap
def bootstrap(target, predict, n):
    profits = []
    count = 0

    for i in range(1000):
        target_subsample = target.reset_index(drop= True).sample(n, random_state=STATE, replace=True)
        probs_subsample = predict[target_subsample.index]
    
        profits.append(profit(target_subsample, probs_subsample, BEST_POINTS))
        
    profits = pd.Series(profits)
    
    count = (profits < 0).mean()

    lower = profits.quantile(0.025)
    upper = profits.quantile(0.975)

    print('Средняя прибыль в регионе равна: %.2f' % (profits.mean()), 'млн. руб.')
    print('95% доверительный интервал лежит в диапазоне: {:.2f}'.format(lower),
          '- {:.2f}'.format(upper))
    print('Вероятность убытка равна: {:.2%}'.format(count))

In [14]:
for i, x in zip(targets_valid, predicts):
    bootstrap(i, x, POINTS)

Средняя прибыль в регионе равна: 394.28 млн. руб.
95% доверительный интервал лежит в диапазоне: -69.45 - 915.50
Вероятность убытка равна: 6.10%
Средняя прибыль в регионе равна: 454.74 млн. руб.
95% доверительный интервал лежит в диапазоне: 61.07 - 855.91
Вероятность убытка равна: 0.70%
Средняя прибыль в регионе равна: 353.66 млн. руб.
95% доверительный интервал лежит в диапазоне: -162.65 - 847.76
Вероятность убытка равна: 7.60%


## Вывод.

В этом исследовании нам были предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Предоставленные данные были отлично подготовлены. Далее мы разбили данные на выборки и обучили модели, сохранив ключевые для исследования значения в списки. По результатам обучения модель, обученная на данных из второй выборки имела самую лучшую точность (99.96%). Объём нефти в каждом регионе заметно ниже, требуемого для безубыточной работы. Следовательно, вести разработку всех скважин в регионах, нерентабельно. 

Произведена оценка по 200 лучшим скважинам в каждом регионе. И по прогнозам 2 регион является самым лучшим для разработки скважин с показателем риска меньше требуемого 2.5% и самой высокой средней прибылью в 450-490 млн. руб. 